In [2]:
import numpy as np

In [22]:
class BackPropagationNetwork:
    """A back-propagation Network"""
    
    #
    # Class members
    #
    layerCount = 0
    shape = None
    weights=[]
    
    #
    # Class methods
    #
    def __init__(self, layerSize):
        """ Initialize the network"""
        
        # layer info
        self.layerCount = len(layerSize) -1
        self.shape = layerSize
        
        # Input/Output data from last Run
        self._layerInput = []
        self._layerOutput = []
        
        # Create the weight arrays
        for (l1,l2) in zip(layerSize[:-1],layerSize[1:]):
            self.weights.append(np.random.normal(scale = 0.1,size=(l2,l1+1)))
            

            
    #
    # Run method
    #
    def Run(self,input):
        """Run the network based on the input data"""
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput=[]
        self._layerOutput=[]
        
        # Run it!
        for index in range(self.layerCount):
            # Determine layer input
            if index ==0:
                layerInput = self.weights[0].dot(np.vstack([input.T,np.ones([1,lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1],np.ones([1,lnCases])]))
                
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.sgm(layerInput))
        return self._layerOutput[-1].T
    
    #
    # TrainEpoch method
    #
    def TrainEpoch(self, input,target, trainingRate = 0.2):
        """This method trains the network for one epoch"""
        
        delta = []
        lnCases = input.shape[0]
        
        # First run the network
        self.Run(input)
        
        # Calculate our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount -1:
                # compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.sgm(self._layerInput[index],True))
            else:
                #Compare to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1,:] * self.sgm(self._layerInput[index],True))
                
            
        # Compute weight deltas
        for index in range(self.layerCount):
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1,lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index -1],np.ones([1,self._layerOutput[index-1].shape[1]])])
                
            weightDelta = np.sum(\
                                layerOutput[None,:,:].transpose(2,0,1) * delta[delta_index][None,:,:].transpose(2,1,0)\
                                ,axis = 0)
                
            self.weights[index] -= trainingRate * weightDelta
            
        return error
    
    # Transfer functions
    def sgm(self,x,Derivative = False):
        if not Derivative:
            return 1 / (1+np.exp(-x))
        else:
            out = self.sgm(x)
            return out*(1-out)
            
#
# if run as a script, create a test object
#
if __name__ == "__main__":
    bpn = BackPropagationNetwork((2,2,1))
    print(bpn.shape)
    print(bpn.weights)
    
    lvInput = np.array([[0,0],[1,1],[0,1],[1,0]])
    lvTarget = np.array([[0.05],[0.05],[0.95],[0.95]])
    
    lnMax = 100000
    lnErr = 1e-5
    for i in range(lnMax+1):
        err = bpn.TrainEpoch(lvInput, lvTarget)
        if i % 2500 == 0:
            print("Iteration {a}\tError: {b:0.6f}".format(a=i, b=err))
        if err <= lnErr:
            print("Desired error reached. Iter: {0}".format(i))
            break
    # Display output
    lvOutput = bpn.Run(lvInput)
    
    print("Input: {0}\nOutput: {1}".format(lvInput,lvOutput))

(2, 2, 1)
[array([[ 0.02634416,  0.07927795, -0.09796164],
       [-0.11855981,  0.00565045, -0.0538474 ]]), array([[ 0.18195193,  0.04031587,  0.0425824 ]])]
Iteration 0	Error: 0.816
Iteration 2500	Error: 0.810
Iteration 5000	Error: 0.810
Iteration 7500	Error: 0.810
Iteration 10000	Error: 0.810
Iteration 12500	Error: 0.810
Iteration 15000	Error: 0.810
Iteration 17500	Error: 0.810
Iteration 20000	Error: 0.810
Iteration 22500	Error: 0.810
Iteration 25000	Error: 0.810
Iteration 27500	Error: 0.810
Iteration 30000	Error: 0.810
Iteration 32500	Error: 0.810
Iteration 35000	Error: 0.810
Iteration 37500	Error: 0.810
Iteration 40000	Error: 0.810
Iteration 42500	Error: 0.810
Iteration 45000	Error: 0.810
Iteration 47500	Error: 0.810
Iteration 50000	Error: 0.810
Iteration 52500	Error: 0.810
Iteration 55000	Error: 0.810
Iteration 57500	Error: 0.810
Iteration 60000	Error: 0.810
Iteration 62500	Error: 0.810
Iteration 65000	Error: 0.810
Iteration 67500	Error: 0.810
Iteration 70000	Error: 0.810
Iterati

In [4]:
A = np.array([[1,1],[2,2]]).T

In [5]:
A[None,:,:]

array([[[1, 2],
        [1, 2]]])

In [6]:
A[None,:,:].transpose()

array([[[1],
        [1]],

       [[2],
        [2]]])

In [8]:
A[None,:,:].transpose(2,0,1)

array([[[1, 1]],

       [[2, 2]]])

In [9]:
A[None,:,:].transpose(2,1,0)

array([[[1],
        [1]],

       [[2],
        [2]]])